In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1.2.7 드롭아웃

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 데이터 구성 및 준비하기
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/NHANES Weight and Height/data.csv')

def BMI_Classification(BMI):
    if BMI <= 18.5:
        return 0 #'저체중'
    elif (18.5 < BMI <= 23):
        return 1 #'정상'
    elif (23 < BMI <= 25):
        return 2 #'과체중'
    elif (25 < BMI <= 30):
        return 3 #'비만'
    elif BMI > 30:
        return 4 #'고도비만'

data['BMI_CF'] = data['BMI(kg/m**2)'].apply(BMI_Classification)

feature_cols = ['Weight (kg)', 'Standing Height (cm)']

label = data.pop('BMI_CF')
feature = data[feature_cols].copy()

# 학습과 테스트 데이터 분리 (학습 80%, 테스트 20%)
train_x, test_x, train_y, test_y = train_test_split(feature, label, test_size=0.2, random_state=0)
# 학습과 검증 데이터 분리 (학습 80%, 검증 20%)
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=0)

print('train dataset :', train_x.shape, train_y.shape)
print('validation dataset :', val_x.shape, val_y.shape)
print('test dataset :', test_x.shape, test_y.shape)

train dataset : (5368, 2) (5368,)
validation dataset : (1342, 2) (1342,)
test dataset : (1678, 2) (1678,)


In [2]:
from tensorflow.keras.callbacks import EarlyStopping
from keras.callbacks import LambdaCallback

# EarlyStopping
early_stopping = EarlyStopping(patience = 20)

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.layers import Dropout, BatchNormalization

tf.random.set_seed(0)

# 모델 생성 
model = Sequential([
    Dense(units=1024, activation='relu', input_shape=[2]),
    Dropout(0.3),
    Dense(units=1024, activation='relu'),
    Dropout(0.3),
    Dense(units=1024, activation='relu'),
    Dropout(0.3),
    Dense(units=5, activation='softmax') # Classification
])

# 모델 컴파일 
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1024)              3072      
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 1024)              1049600   
                                                                 
 dropout_1 (Dropout)         (None, 1024)              0         
                                                                 
 dense_2 (Dense)             (None, 1024)              1049600   
                                                                 
 dropout_2 (Dropout)         (None, 1024)              0         
                                                                 
 dense_3 (Dense)             (None, 5)                 5

In [4]:
history = model.fit(train_x, train_y,
                    validation_data=(val_x, val_y),
                    epochs=1000,
                    callbacks=early_stopping)

Epoch 1/1000
168/168 [==============================] - 8s 42ms/step - loss: 1.6083 - accuracy: 0.6384 - val_loss: 0.5361 - val_accuracy: 0.7623
Epoch 2/1000
168/168 [==============================] - 7s 40ms/step - loss: 0.6841 - accuracy: 0.7109 - val_loss: 0.5353 - val_accuracy: 0.7824
Epoch 3/1000
168/168 [==============================] - 7s 40ms/step - loss: 0.6230 - accuracy: 0.7284 - val_loss: 0.4842 - val_accuracy: 0.7802
Epoch 4/1000
168/168 [==============================] - 7s 40ms/step - loss: 0.5894 - accuracy: 0.7494 - val_loss: 0.4898 - val_accuracy: 0.7951
Epoch 5/1000
168/168 [==============================] - 8s 50ms/step - loss: 0.5821 - accuracy: 0.7474 - val_loss: 0.4758 - val_accuracy: 0.7884
Epoch 6/1000
168/168 [==============================] - 13s 78ms/step - loss: 0.5552 - accuracy: 0.7615 - val_loss: 0.4883 - val_accuracy: 0.7914
Epoch 7/1000
168/168 [==============================] - 13s 79ms/step - loss: 0.5371 - accuracy: 0.7617 - val_loss: 0.5608 - val_

In [6]:
import plotly.graph_objects as go

fig = go.Figure( )
fig.add_trace(go.Scattergl(y=history.history['loss'], name='loss')) 
fig.add_trace(go.Scattergl(y=history.history['val_loss'], name='val_loss')) 
fig.update_layout(title="<b>Loss of Model</b>", xaxis_title='Epoch', yaxis_title='Loss', template='plotly')
fig.show( )

In [ ]:
model.evaluate(test_x, test_y)

53/53 [==============================] - 0s 6ms/step - loss: 1.2792 - accuracy: 0.6311


[1.2791504859924316, 0.6311084628105164]